This file is the combination for all the cut I use for cutting the comics muon events. The comics muon cut use the data produced by muon skim. The cuts is writed base on following idea. 

1. There are more than one pulse in the event is greater than
2. The most of pulses in the event should happen in the same layer. For example, if the 2/3 pulses in a event is happened in the same layer, the event is selected.
3. Time cutting, the time difference between hits at different chanel is in 20ns. Since the calibration still haven't finish, the cut does not consider the order of time and its layers. 

This notebook is changed base on Colin's notebook for beam muon analysis.

In [3]:
import ROOT as r
from ROOT import TFile, TTree, TH1F, TChain,TCanvas
from ROOT import *
import os
import sys
import time
import numpy as np


Welcome to JupyROOT 6.24/07


In [4]:
class DataHandler():
    def __init__(self, dataPath, debug=False):
        self.data = self.initializeData(dataPath)
        self.debug = debug
    # Create TChain with all data you will need for your analysis
    # wildcards can be used in datapath
    def initializeData(self, datapath):
        treeChain = r.TChain('t')
        treeChain.Add(datapath)
        return treeChain
    
    def cosmic(self,event):
        row_list = event.row
        column_list = event.column
        layer_list = event.layer
        height_list = event.height
        DAQNum = event.DAQEventNumber
        chan_list = event.chan
        time_list = event.timeFit
        
        #delete small peak in channel 0 
        for chan,height in zip(chan_list,height_list):
            if chan == 0 and height > 1000: return None
            else: continue
               
        #check the pulse has 500mv height more than 1
        i = 0
        for height in height_list:
            if height > 500:
                i += 1
        if i < 2: return

        m, n, k = 4, 4, 4
        zero_matrix = [[[0 for k in range(k)] for j in range(n)] for i in range(m)]
        
        for row,column,layer in zip(row_list,column_list,layer_list):
            zero_matrix[row][layer][column] = 1
            

            
        
        #check there is no ajacent element in each layer
        for k in range(4):
            for i in range(4):
                for j in range(4):
                    if j < 3 and zero_matrix[k][i][j] == zero_matrix[k][i][j+1] and zero_matrix[k][i][j] == 1: return 
                    
                    if i < 3 and zero_matrix[k][i][j] == zero_matrix[k][i+1][j] and zero_matrix[k][i+1][j] == 1: return
                    
                    else: continue
        

        
        #This part do the cut for the event do not content the rows of 1-3, that is order 3-1
        row_list_cut = [3,2,1]
        row_check = []
        i = 0
        
        for x in row_list:
            if x in row_list_cut:
                row_check.append(x)
            
                
        if len(row_check)==0: return 
        
        
        
        # Delete the muon which could not be cosmic muon by checking the layer
        layer1 = []
        layer2 = []
        layer3 = []
        layer4 = []
        for layer in layer_list:
            if layer == 0:
                layer1.append(layer)
            elif layer == 1:
                layer2.append(layer)
            elif layer == 2:
                layer3.append(layer)
            elif layer == 3:
                layer4.append(layer)
        if len(layer1) < 2/3*len(layer_list) and len(layer2) < 2/3*len(layer_list) and len(layer3) < 2/3*len(layer_list) and len(layer4) < 2/3*len(layer_list): 
            return 


       
        #check the time difference of pulse height than 500mv is less than 20ns 
        time_pulse_list = []
        channel_pulse_list = []
    
        for iheight, height in enumerate(height_list):
            if height > 500:
                time_pulse_list.append(time_list[iheight])
                channel_pulse_list.append(chan_list[iheight])
        max_time = max(time_pulse_list)
        min_time = min(time_pulse_list)
        max_channel = channel_pulse_list[time_pulse_list.index(max_time)]
        min_channel = channel_pulse_list[time_pulse_list.index(min_time)]
        
        if max_time-min_time <= 20 and max_channel != min_channel:
            return event


    def applyCuts(self, cuts):
        lc=len(cuts)
        cutData = [] 
        pulse_height_list = []
        

        for event in self.data:
            None_check = []
            for cut in cuts:
                
                event= cut(event) # Apply given cut
                if event is None: 
                    break
                None_check.append(event.DAQEventNumber)
                

            if None in None_check:
                break
            if len(None_check) == lc: # if there are two cuts then len(None_check) should be 2
                cutData.append(None_check[0])
                pulse_height_list.append(np.array(event.height))
            
        return cutData, pulse_height_list

In [5]:
if __name__ == "__main__":
    data = DataHandler('/store/user/mcarrigan/skim/muon/v29/MilliQan_Run591.*_MuonSkim_v29.root')
    cutData, pulse_height_list = data.applyCuts([data.cosmic])
    print(len(cutData))
    

KeyboardInterrupt: 